# Data Engineering

## Transactions with delivery information

#### 1. load packages

In [16]:
library(data.table)
library(dplyr)
library(ggplot2)
library(lubridate)
options(warn=-1)

#### 2. set working directory and load data

In [18]:
setwd("/Users/sunmengnan/Documents/GitHub/thgfd/data")
trans <- fread("MAIN_transaction_data.csv")
fraud_trans <- fread("MAIN_chargeback_data.csv")
customers <- read.csv("MAIN_customer_data.csv")

#### 3. read delivery key file and change the column name to match main transaction file

In [19]:
del_key <- fread("delivery_option_lookup.csv")
colnames(del_key) <- c("Delivery_Option_Type_Key", "Name")

#### 4. Combine the delivery file and main transaction file together by "Name", Creat a new dataframe, calculate the times of Delivery name in the main transction

In [21]:
trans_del <- trans %>% 
  left_join(del_key) %>% 
  group_by(Name) %>% 
  summarise(n = n()) 
trans_del

Joining, by = "Delivery_Option_Type_Key"


Name,n
Air Express Delivery,223
International Standard,137528
International Tracked,43539
Next Day,3724
Premium Express Delivery,10206
Saturday,362
Standard,222377
Sunday,280
Unknown,95


#### 5. Add a new column to the main transaction table by joining the two tables together

In [22]:
trans <- trans %>% 
  left_join(del_key)
head(trans)

Joining, by = "Delivery_Option_Type_Key"


Order_Number,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,Locale_Key,Order_Sequence_No,Medium_Key,Campaign_Key,Order_Payment_Status_Key,⋯,Cancelled_Qty,Cancelled_Date_Key,customer_ip_address,Postcode_Shipping_Address,Country_Code_Shipping_Address,Postcode_Billing_Address,Country_Code_Billing_Address,Empty1,Empty2,Name
67028670,19534992,5,20160301,7,9,1,48,4659,0,⋯,0,-1,121.223.71.121,4814,AU,4814,AU,,,International Standard
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,0,-1,80.2.92.22,Cf54ts,GB,Cf54ts,GB,,,Standard
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,0,-1,80.2.92.22,Cf54ts,GB,Cf54ts,GB,,,Standard
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,0,-1,80.2.92.22,Cf54ts,GB,Cf54ts,GB,,,Standard
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,0,-1,80.2.92.22,Cf54ts,GB,Cf54ts,GB,,,Standard
67085417,19535579,5,20160301,67,12,1,48,123588,0,⋯,0,-1,99.47.47.34,92806,US,92806,US,,,International Standard


#### 6. Create a new dataframe named del_features, we classifiy the destination by international or domestic destinatino, and the priotity, here we assume that if they wanted it delivered only on Saturday/Sunday, it was most likely the case that they were going to be working

In [28]:
del_features <- data.frame(Name = as.character(unique(trans$Name)), 
                           destination = c(rep("int", 4), rep("dom", 5)),
                           priority = c("priority", rep("slow", 2), rep("priority", 2), rep("slow", 4)),
                           occupation = c(rep("unknown", 7), "working", "working"))
del_features

Name,destination,priority,occupation
International Standard,int,priority,unknown
Standard,int,slow,unknown
Next Day,int,slow,unknown
International Tracked,int,priority,unknown
Premium Express Delivery,dom,priority,unknown
Air Express Delivery,dom,slow,unknown
Unknown,dom,slow,unknown
Saturday,dom,slow,working
Sunday,dom,slow,working


#### 7. Combine the new dataframe del_features with the main transaction table through key column  "Name"

In [32]:
trans <- trans %>% 
  left_join(del_features, by = "Name")
head(trans)

Order_Number,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,Locale_Key,Order_Sequence_No,Medium_Key,Campaign_Key,Order_Payment_Status_Key,⋯,Country_Code_Billing_Address,Empty1,Empty2,Name,destination.x,priority.x,occupation.x,destination.y,priority.y,occupation.y
67028670,19534992,5,20160301,7,9,1,48,4659,0,⋯,AU,,,International Standard,int,priority,unknown,int,priority,unknown
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,GB,,,Standard,int,slow,unknown,int,slow,unknown
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,GB,,,Standard,int,slow,unknown,int,slow,unknown
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,GB,,,Standard,int,slow,unknown,int,slow,unknown
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,GB,,,Standard,int,slow,unknown,int,slow,unknown
67085417,19535579,5,20160301,67,12,1,48,123588,0,⋯,US,,,International Standard,int,priority,unknown,int,priority,unknown


## Reliable Customers

#### 1. Count the times of each customer's transaction in main transaction table in a new data frame named customers

In [33]:
customers <- trans %>% 
  group_by(Account_Key) %>% 
  mutate(count = n()) %>% 
  group_by(count) %>% 
  summarise(n = n()) 
head(customers)

count,n
1,79487
2,57252
3,42192
4,39276
5,25175
6,22302


#### 2. Create a new colomun which indicates the proportion of a customer's transaction times in total transaction time

In [35]:
customers$prop <- customers$n/dim(trans)[1] 
head(customers)

count,n,prop
1,79487,0.19000846
2,57252,0.13685715
3,42192,0.10085721
4,39276,0.09388670
5,25175,0.06017919
6,22302,0.05331147


#### 3. Add a new variable which cumulates the percentage of previous top customers

In [36]:
customers$cumulate <- cumsum(customers$prop)
head(customers)

count,n,prop,cumulate
1,79487,0.19000846,0.1900085
2,57252,0.13685715,0.3268656
3,42192,0.10085721,0.4277228
4,39276,0.09388670,0.5216095
5,25175,0.06017919,0.5817887
6,22302,0.05331147,0.6351002


#### 4. Add new variables including pay_key_num(numeric), num_valid, proportion and status indicating whethere the transcation is frand, here we assume that if the total counts of payment status which is known are larger than 2 or the proportion of valid number is less than 25%, it's quesionable, accordingly, we define the insufficient, adeuate and fine type.

In [40]:
payment_hists <- trans %>% 
  group_by(Account_Key) %>% 
  mutate(pay_key_num = as.numeric(Order_Payment_Status_Key),
         num_valid = length(which(pay_key_num == 0)),
         count = n(),
         prop = num_valid/count,
         status = ifelse(prop <= 0.25 & count > 2, "questionable",
                         ifelse(prop <= 0.25 & count <= 2, "insufficient",
                                ifelse(prop > 0.25 & prop < 0.75, "adequate", "fine"))))
head(payment_hists)

Order_Number,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,Locale_Key,Order_Sequence_No,Medium_Key,Campaign_Key,Order_Payment_Status_Key,⋯,priority.x,occupation.x,destination.y,priority.y,occupation.y,pay_key_num,num_valid,count,prop,status
67028670,19534992,5,20160301,7,9,1,48,4659,0,⋯,priority,unknown,int,priority,unknown,0,1,1,1,fine
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,slow,unknown,int,slow,unknown,0,4,4,1,fine
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,slow,unknown,int,slow,unknown,0,4,4,1,fine
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,slow,unknown,int,slow,unknown,0,4,4,1,fine
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,slow,unknown,int,slow,unknown,0,4,4,1,fine
67085417,19535579,5,20160301,67,12,1,48,123588,0,⋯,priority,unknown,int,priority,unknown,0,1,1,1,fine
67094633,10805060,1,20160301,9,3,9,59,125903,0,⋯,slow,unknown,int,slow,unknown,0,3,3,1,fine
67094633,10805060,1,20160301,9,3,9,59,125903,0,⋯,slow,unknown,int,slow,unknown,0,3,3,1,fine
67094633,10805060,2,20160301,9,3,9,59,125903,0,⋯,slow,unknown,int,slow,unknown,0,3,3,1,fine
67095935,19534656,1,20160301,-1,3,1,106,2,0,⋯,slow,unknown,int,slow,unknown,0,5,5,1,fine


#### 5. Based on the payment_hists, we add a new feature called cancelled proportion which is cancelled quantity divided by order quantity

In [42]:
cancelled_items <- payment_hists %>% 
  group_by(Category_Level_3, Category_Level_2) %>% 
  mutate(canc_prop = ifelse(Cancelled_Qty == 0, 0, Cancelled_Qty/Ordered_Qty))
head(cancelled_items)

Order_Number,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,Locale_Key,Order_Sequence_No,Medium_Key,Campaign_Key,Order_Payment_Status_Key,⋯,occupation.x,destination.y,priority.y,occupation.y,pay_key_num,num_valid,count,prop,status,canc_prop
67028670,19534992,5,20160301,7,9,1,48,4659,0,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67085417,19535579,5,20160301,67,12,1,48,123588,0,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0


#### 6. If the order is cancel, we assume it's quesionable and we should check the billing address, to see if it matches the shipping address

In [45]:
bill_ship_match <- cancelled_items %>% 
  mutate(status = ifelse(Postcode_Billing_Address != Postcode_Shipping_Address, "mismatch", "fine"))
head(bill_ship_match)
trans <- bill_ship_match

Order_Number,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,Locale_Key,Order_Sequence_No,Medium_Key,Campaign_Key,Order_Payment_Status_Key,⋯,occupation.x,destination.y,priority.y,occupation.y,pay_key_num,num_valid,count,prop,status,canc_prop
67028670,19534992,5,20160301,7,9,1,48,4659,0,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,19535774,1,20160301,11,3,1,106,2,0,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67085417,19535579,5,20160301,67,12,1,48,123588,0,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0


#### 7. Based on the billing and shipping mismatched table, we extract the related transaction date

In [47]:
trans$Order_Date_Key <- ymd(trans$Order_Date_Key)
fraud_accounts <- customers
colnames(fraud_trans)[1] <- "Order_Number"
head(trans$Order_Date_Key )
head(fraud_accounts)

[1] "2016-03-01" "2016-03-01" "2016-03-01" "2016-03-01" "2016-03-01"
[6] "2016-03-01"

count,n,prop,cumulate
1,79487,0.19000846,0.1900085
2,57252,0.13685715,0.3268656
3,42192,0.10085721,0.4277228
4,39276,0.09388670,0.5216095
5,25175,0.06017919,0.5817887
6,22302,0.05331147,0.6351002


#### 8. Now we connect that table the fraud transaction table together by order number

In [48]:
fraud <- fraud_trans %>% 
  inner_join(trans, by = "Order_Number")

#### 9. Extract the date which is missing in hte fraud transaction table, using the joint previous tables

In [49]:
to_app <- fraud_trans %>% 
  right_join(trans, by = "Order_Number") %>% 
  filter(is.na(Internal.RC) == TRUE)
head(to_app)

Order_Number,Date.Logged,Released_by,Internal.RC,Defence.Status,GBP.Amount,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,⋯,occupation.x,destination.y,priority.y,occupation.y,pay_key_num,num_valid,count,prop,status,canc_prop
67028670,NA,NA,NA,NA,NA,19534992,5,2016-03-01,7,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,unknown,int,slow,unknown,0,4,4,1,fine,0
67085417,NA,NA,NA,NA,NA,19535579,5,2016-03-01,67,⋯,unknown,int,priority,unknown,0,1,1,1,fine,0


#### 10. Bind the two tables together and create a new variable to show whether it is fraud

In [50]:
trans <- rbindlist(list(to_app, fraud))
trans$fraud_status <- ifelse(is.na(trans$Internal.RC) == TRUE, 0, 1)
head(trans)

Order_Number,Date.Logged,Released_by,Internal.RC,Defence.Status,GBP.Amount,Account_Key,Delivery_Option_Type_Key,Order_Date_Key,Payment_Method_Key,⋯,destination.y,priority.y,occupation.y,pay_key_num,num_valid,count,prop,status,canc_prop,fraud_status
67028670,NA,NA,NA,NA,NA,19534992,5,2016-03-01,7,⋯,int,priority,unknown,0,1,1,1,fine,0,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,int,slow,unknown,0,4,4,1,fine,0,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,int,slow,unknown,0,4,4,1,fine,0,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,int,slow,unknown,0,4,4,1,fine,0,0
67077956,NA,NA,NA,NA,NA,19535774,1,2016-03-01,11,⋯,int,slow,unknown,0,4,4,1,fine,0,0
67085417,NA,NA,NA,NA,NA,19535579,5,2016-03-01,67,⋯,int,priority,unknown,0,1,1,1,fine,0,0


#### 11. Extract the informaion we want, exculding order number, realeased_by, internal.RC, defence.statues, GBP.Amount, accout_key, empty1 and empty2 columns

In [51]:
trans <- select(trans,-c(Order_Number, Released_by, Internal.RC, Defence.Status, GBP.Amount, Account_Key, Empty1, Empty2))
write.csv(trans, "notFinished_transactions_updated.csv")